<a href="https://colab.research.google.com/github/madhvv/learning-pytorch/blob/main/Number_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.52MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 130kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.24MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.2MB/s]


In [3]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [4]:
from torch.utils.data import DataLoader

loaders = {
    'train' : DataLoader(
        train_data,
        batch_size = 100,
        shuffle = True,
        num_workers = 1
    ),

    'test' : DataLoader(
        test_data,
        batch_size = 100,
        shuffle = True,
        num_workers = 1
    )
}

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()

    self.conv1 = nn.Conv2d(1,10,kernel_size=5)
    self.conv2 = nn.Conv2d(10,20,kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320,50)
    self.fc2 = nn.Linear(50,10)


  def forward(self,x):
    x = F.relu(F.max_pool2d(self.conv1(x),2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
    x = x.view(-1,320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x,training = self.training)
    x = self.fc2(x)

    return F.softmax(x)


In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
  model.train()
  for batch_idx, (data,target) in enumerate(loaders['train']):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    output = model(data)
    loss = loss_fn(output,target)
    loss.backward()
    optimizer.step()

    if batch_idx % 20 == 0:
      print(f'Train Epoch : {epoch} [{batch_idx* len(data)}/{len(loaders["train"].dataset)} ({100.*batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
  model.eval()

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in loaders['test']:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_fn(output, target).item()
      pred = output.argmax(dim = 1, keepdim = True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(loaders['test'].dataset)
  print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)')



In [10]:
for epoch in range(1,11):
  train(epoch)
  test()


/tmp/ipython-input-3869106656.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch : 1 [0/60000 (0%)]	2.302323
Train Epoch : 1 [2000/60000 (3%)]	2.292770
Train Epoch : 1 [4000/60000 (7%)]	2.212537
Train Epoch : 1 [6000/60000 (10%)]	2.044286
Train Epoch : 1 [8000/60000 (13%)]	1.839537
Train Epoch : 1 [10000/60000 (17%)]	1.884382
Train Epoch : 1 [12000/60000 (20%)]	1.797029
Train Epoch : 1 [14000/60000 (23%)]	1.802042
Train Epoch : 1 [16000/60000 (27%)]	1.702760
Train Epoch : 1 [18000/60000 (30%)]	1.740782
Train Epoch : 1 [20000/60000 (33%)]	1.738241
Train Epoch : 1 [22000/60000 (37%)]	1.799588
Train Epoch : 1 [24000/60000 (40%)]	1.738204
Train Epoch : 1 [26000/60000 (43%)]	1.727942
Train Epoch : 1 [28000/60000 (47%)]	1.685102
Train Epoch : 1 [30000/60000 (50%)]	1.696017
Train Epoch : 1 [32000/60000 (53%)]	1.657773
Train Epoch : 1 [34000/60000 (57%)]	1.763460
Train Epoch : 1 [36000/60000 (60%)]	1.725321
Train Epoch : 1 [38000/60000 (63%)]	1.685972
Train Epoch : 1 [40000/60000 (67%)]	1.768567
Train Epoch : 1 [42000/60000 (70%)]	1.593419
Train Epoch : 1 [4400